# Crypto Backtesting – Full ML & DL Stack

This notebook trains multiple ML/DL models (RandomForest, XGBoost, AdaBoost, MLP, HMM regime, CNN, LSTM, GRU) on minute‑level OHLCV data for multiple cryptocurrencies and evaluates each with **Backtesting.py**.

Adjust `CSV_PATHS` and hyper‑parameters to your liking, then run all cells.

In [1]:
!pip -q install pandas numpy ta backtesting scikit-learn xgboost hmmlearn tensorflow matplotlib

In [2]:
import pandas as pd, numpy as np
import sklearn
import warnings
import tensorflow
from pathlib import Path
from ta.volatility import BollingerBands
from ta.trend import SMAIndicator
from ta.momentum import RSIIndicator, StochasticOscillator
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from hmmlearn.hmm import GaussianHMM
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Conv1D, LSTM, GRU, Dense, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from backtesting import Backtest, Strategy
import matplotlib.pyplot as plt

pd.set_option('display.width', 140)

/Users/jadenfix/.pyenv/versions/3.11.3/lib/python3.11/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

## 1  Load and feature‑engineer data

In [3]:
CSV_PATHS = {
    'BTC': Path('/Users/jadenfix/hedge-fund-in-a-box/cpp_engine/data/2024_2025/2024_to_april_2025_btc_data.csv'),
    'ETH': Path('/Users/jadenfix/hedge-fund-in-a-box/cpp_engine/data/2024_2025/2024_to_april_2025_eth_data.csv'),
    'SOL': Path('/Users/jadenfix/hedge-fund-in-a-box/cpp_engine/data/2024_2025/2024_to_april_2025_solana_data.csv'),
    'ADA': Path('/Users/jadenfix/hedge-fund-in-a-box/cpp_engine/data/2024_2025/2024_to_april_2025_ada_data.csv')
}


def load_crypto(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    # take only the first 10k rows
    df = df.iloc[:10_000]
    df['datetime'] = pd.to_datetime(df['date_only'] + ' ' + df['time_only'])
    df = df.set_index('datetime').sort_index()
    df = df.rename(columns=str.lower)[['open','high','low','close','volume']]
    return df

def add_indicators(df):
    c = df['close']
    df['sma_short'] = SMAIndicator(c, 14).sma_indicator()
    df['sma_long']  = SMAIndicator(c, 50).sma_indicator()
    df['sma_ratio'] = df['sma_short'] / df['sma_long']
    df['rsi']       = RSIIndicator(c, 14).rsi()
    bb = BollingerBands(c, 20, 2)
    df['bb_high']   = bb.bollinger_hband()
    df['bb_low']    = bb.bollinger_lband()
    df['bb_width']  = (df['bb_high'] - df['bb_low']) / c
    stoch = StochasticOscillator(df['high'], df['low'], c, 14)
    df['stoch'] = stoch.stoch()
    df['return'] = c.pct_change().shift(-1)
    df['volatility'] = c.rolling(window=14).std()
    df['direction'] = np.sign(df['return']).replace(0, np.nan).bfill()
    return df.dropna()

# Process all cryptocurrencies
crypto_data = {coin: add_indicators(load_crypto(path)) for coin, path in CSV_PATHS.items()}

## 2  Train/test split with cross-validation

In [4]:
FEATURES = ['sma_short', 'sma_long', 'sma_ratio', 'rsi', 'bb_high', 'bb_low', 'bb_width', 'stoch', 'volatility']

def prepare_data(df):
    df_bt = df.rename(columns={'open':'Open','high':'High','low':'Low','close':'Close','volume':'Volume'}).dropna()
    X, y = df[FEATURES], df['direction']
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, y

# Prepare data for each cryptocurrency
crypto_datasets = {coin: prepare_data(data) for coin, data in crypto_data.items()}

## 2.1

In [5]:
def prepare_data(df):
    df_bt = df.rename(columns={'open':'Open','high':'High',
                               'low':'Low','close':'Close',
                               'volume':'Volume'}).dropna()

    X = df[FEATURES]
    # ← here we remap –1→0, +1→1
    y = df['direction'].map({-1.0: 0, 1.0: 1})

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, y

# Now crypto_datasets will contain y in {0,1}
crypto_datasets = {coin: prepare_data(data)
                   for coin, data in crypto_data.items()}

## 3  Fit classical ML models with robust cross-validation

In [6]:
# 1) Parameter grids for each model
param_grids = {
    'RF': {
        'n_estimators': [100, 300, 500],
        'max_depth':    [6, 10, None],
        'class_weight':['balanced'],
        'random_state':[42]
    },
    'XGB': {
        'n_estimators':     [100, 250, 500],
        'max_depth':        [3, 4, 6],
        'learning_rate':    [0.01, 0.05, 0.1],
        'subsample':        [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0],
        'random_state':     [42]
    },
    'ADB': {
        'n_estimators':  [50, 100, 200],
        'learning_rate': [0.1, 0.5],
        'random_state':  [42]
    },
    'MLP': {
        'hidden_layer_sizes': [(64, 32), (100, 50)],
        'max_iter':           [500, 1000],
        'tol':                [1e-3, 1e-4],
        'batch_size':         [128],
        'solver':             ['adam'],
        'random_state':       [42]
    }
}

# 2) TimeSeriesSplit to respect temporal ordering
tscv = TimeSeriesSplit(n_splits=5)

# 3) Mapping of model names to their classes
model_classes = {
    'RF':  RandomForestClassifier,
    'XGB': XGBClassifier,
    'ADB': AdaBoostClassifier,
    'MLP': MLPClassifier
}

# 4) Run GridSearchCV for each coin & each model
best_models = {}

for coin, (X, y) in crypto_datasets.items():
    print(f"\n=== Grid search for {coin} ===")
    best_models[coin] = {}
    
    for name, ModelClass in model_classes.items():
        print(f"\n>> Tuning {name}...")
        
        gs = GridSearchCV(
            estimator   = ModelClass(),
            param_grid  = param_grids[name],
            cv          = tscv,
            scoring     = 'balanced_accuracy',
            n_jobs      = -1,
            verbose     = 2
        )
        gs.fit(X, y)
        
        print(f"✔ Best {name} params: {gs.best_params_}")
        print(f"✔ Best CV score:   {gs.best_score_:.4f}")
        
        best_models[coin][name] = gs.best_estimator_
        crypto_models = best_models
# Now best_models['BTC']['MLP'], etc., hold your tuned classifiers.


=== Grid search for BTC ===

>> Tuning RF...
Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END class_weight=balanced, max_depth=6, n_estimators=100, random_state=42; total time=   0.5s
[CV] END class_weight=balanced, max_depth=6, n_estimators=100, random_state=42; total time=   0.9s
[CV] END class_weight=balanced, max_depth=6, n_estimators=100, random_state=42; total time=   1.3s
[CV] END class_weight=balanced, max_depth=6, n_estimators=300, random_state=42; total time=   1.4s
[CV] END class_weight=balanced, max_depth=6, n_estimators=100, random_state=42; total time=   1.7s
[CV] END class_weight=balanced, max_depth=6, n_estimators=100, random_state=42; total time=   2.1s
[CV] END class_weight=balanced, max_depth=6, n_estimators=300, random_state=42; total time=   2.5s
[CV] END class_weight=balanced, max_depth=6, n_estimators=500, random_state=42; total time=   2.5s
[CV] END class_weight=balanced, max_depth=6, n_estimators=300, random_state=42; total time=   3.9s
[CV

/Users/jadenfix/.pyenv/versions/3.11.3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END batch_size=128, hidden_layer_sizes=(64, 32), max_iter=500, random_state=42, solver=adam, tol=0.0001; total time=   4.4s
[CV] END batch_size=128, hidden_layer_sizes=(64, 32), max_iter=500, random_state=42, solver=adam, tol=0.0001; total time=   5.3s
[CV] END batch_size=128, hidden_layer_sizes=(64, 32), max_iter=1000, random_state=42, solver=adam, tol=0.0001; total time=   4.4s
[CV] END batch_size=128, hidden_layer_sizes=(64, 32), max_iter=1000, random_state=42, solver=adam, tol=0.0001; total time=   4.7s
[CV] END batch_size=128, hidden_layer_sizes=(100, 50), max_iter=500, random_state=42, solver=adam, tol=0.001; total time=   0.5s
[CV] END batch_size=128, hidden_layer_sizes=(100, 50), max_iter=500, random_state=42, solver=adam, tol=0.001; total time=   1.0s
[CV] END batch_size=128, hidden_layer_sizes=(100, 50), max_iter=500, random_state=42, solver=adam, tol=0.001; total time=   0.8s
[CV] END batch_size=128, hidden_layer_sizes=(100, 50), max_iter=500, random_state=42, solver=ad

## 4  HMM bull/bear regime

In [7]:
def train_hmm(df,
              n_states: int = 2,
              n_iter: int = 1000,      # ↑ give it lots more iterations
              tol: float   = 1e-4,     # stop when improvements < tol
              verbose: bool = True):   # print log-likelihood each step

    # reshape to shape (n_samples, n_features)
    X = df[['return']].values

    model = GaussianHMM(
        n_components   = n_states,
        covariance_type= 'full',
        n_iter         = n_iter,
        tol            = tol,
        verbose        = verbose,
        init_params    = 'stmcw',    # initialize startprob, transmat, means, covars, weights
        random_state   = 42
    )

    model.fit(X)

    states = model.predict(X)
    # map the first state to +1 and the other to –1
    first, second = states[0], 1 - states[0]
    signals = pd.Series(states, index=df.index).map({ first:  1,
                                                     second: -1 })
    df['hmm_signal'] = signals

    return model, states

# retrain for each coin
crypto_hmm = { coin: train_hmm(df) 
               for coin, df in crypto_data.items() }

python(58595) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
         1   25213.09608910             +nan
         2   54237.12268609  +29024.02659700
         3   55420.64222837   +1183.51954228
         4   55820.00718298    +399.36495461
         5   55966.96847905    +146.96129607
         6   55984.48370659     +17.51522754
         7   55978.05336430      -6.43034229
Model is not converging.  Current: 55978.053364303225 is not greater than 55984.48370659294. Delta is -6.430342289713735
         1   25213.83397051             +nan
         2   54343.14410887  +29129.31013836
         3   55431.10307710   +1087.95896823
         4   55827.06816459    +395.96508749
         5   55980.09201876    +153.02385417
         6   56007.32955803     +27.23753927
         7   56005.95826656      -1.37129147
Model is not converging.  Current: 56005.958266558795 is not greater than 56007.3295580297. Delta is -1.3712914709030883
         1   25170.87942470   

## 5  Deep Learning models with early stopping

In [8]:
WINDOW = 20   # number of past timesteps fed to the network

# ------------------------------------------------------------------
# 1) reshape each (X, y) pair into sliding‑window tensors for DL
# ------------------------------------------------------------------
def prepare_dl_data(X: np.ndarray, y):
    X = np.asarray(X, dtype=np.float32)
    y = np.asarray(y, dtype=np.float32).reshape(-1, 1)

    X_dl = np.array([X[i : i + WINDOW] for i in range(len(X) - WINDOW)], dtype=np.float32)
    y_dl = y[WINDOW:]            # align target with last row of each window
    return X_dl, y_dl

# ------------------------------------------------------------------
# 2) model builders
# ------------------------------------------------------------------
def make_cnn(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(32, 3, activation='relu'),
        Conv1D(16, 3, activation='relu'),
        Flatten(),
        Dense(16, activation='relu'),
        Dense(1,  activation='tanh')
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

def make_rnn(rnn_layer, input_shape):
    model = Sequential([
        Input(shape=input_shape),
        rnn_layer(32, return_sequences=False),
        Dense(16, activation='relu'),
        Dense(1,  activation='tanh')
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# ------------------------------------------------------------------
# 3) train a CNN, LSTM, and GRU for every coin
# ------------------------------------------------------------------
crypto_dl_models = {}

for coin, (X, y) in crypto_datasets.items():
    X_dl, y_dl = prepare_dl_data(X, y)
    input_shape = (WINDOW, X_dl.shape[2])   # (timesteps, n_features)

    crypto_dl_models[coin] = {
        'CNN':  make_cnn(input_shape),
        'LSTM': make_rnn(LSTM, input_shape),
        'GRU':  make_rnn(GRU,  input_shape),
    }

    for name, model in crypto_dl_models[coin].items():
        print(f"\nTraining {coin} – {name}")
        model.fit(
            X_dl, y_dl,
            epochs=50,
            batch_size=128,
            validation_split=0.20,   # enables val_loss for early‑stopping
            callbacks=[early_stopping],
            verbose=1
        )


Training BTC – CNN
Epoch 1/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3137 - val_loss: 0.2563
Epoch 2/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2572 - val_loss: 0.2549
Epoch 3/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2546 - val_loss: 0.2555
Epoch 4/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2531 - val_loss: 0.2532
Epoch 5/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2490 - val_loss: 0.2528
Epoch 6/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2479 - val_loss: 0.2537
Epoch 7/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2490 - val_loss: 0.2535
Epoch 8/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2459 - val_loss: 0.2529
Epoch 9/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2461 - val_loss: 0.2544
Epoch 10/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2446 - val_loss: 0.2550

Training BTC – LSTM
Epoch 1/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.3165 - val_loss: 0.2714
Epoch 2/50
63/63 ━━━━━━━━━━━━━━━━

## 6  Backtesting strategies with comprehensive performance tracking

In [ ]:
class MLStrategy(Strategy):
    model = None
    feats = FEATURES
    window = WINDOW
    def init(self): pass
    def _predict(self):
        if isinstance(self.model, tf.keras.Model):
            if len(self.data) < self.window: return 0
            x = self.data.df[self.feats].iloc[-self.window:].values
            return float(self.model.predict(x[np.newaxis], verbose=0)[0,0])
        elif self.model is None:
            return self.data.df['hmm_signal'].iloc[-1]
        else:
            row = [self.data.df[f].iloc[-1] for f in self.feats]
            return self.model.predict([row])[0]
    def next(self):
        sig = np.sign(self._predict())
        if sig > 0 and not self.position.is_long:
            self.position.close(); self.buy()
        elif sig < 0 and not self.position.is_short:
            self.position.close(); self.sell()

# Comprehensive backtesting results
results = {}
for coin, data in crypto_data.items():
    df_bt = data.rename(columns={'open':'Open','high':'High','low':'Low','close':'Close','volume':'Volume'}).dropna()
    results[coin] = {}
    
    # Classical ML models
    for name, mdl in crypto_models[coin].items():
        Strat = type(f'{name}Strat', (MLStrategy,), {'model': mdl})
        stats = Backtest(df_bt, Strat, cash=10_000_000, commission=.001, exclusive_orders=True).run()
        results[coin][name] = stats
    
    # Deep Learning models
    for name, mdl in crypto_dl_models[coin].items():
        Strat = type(f'{name}Strat', (MLStrategy,), {'model': mdl})
        stats = Backtest(df_bt, Strat, cash=10_000_000, commission=.001, exclusive_orders=True).run()
        results[coin][name] = stats
    
    # HMM Strategy
    Strat = type('HMMStrat', (MLStrategy,), {'model': None})
    stats = Backtest(df_bt, Strat, cash=10_000_000, commission=.001, exclusive_orders=True).run()
    results[coin]['HMM'] = stats

# Print and visualize results
for coin, coin_results in results.items():
    print(f"\n{coin} Cryptocurrency Results:")
    for name, stats in coin_results.items():
        print(f"{name:5s}: Return {stats['Return [%]']:.2f}%  |  Equity ${stats['_equity_final']:.0f}")

# Optional: Create a comparative visualization
plt.figure(figsize=(15, 10))
for coin, coin_results in results.items():
    returns = [stats['Return [%]'] for stats in coin_results.values()]
    plt.bar([f"{coin} - {name}" for name in coin_results.keys()], returns)
plt.title('Model Performance Across Cryptocurrencies')
plt.ylabel('Return [%]')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()